## Respuesta dinámica a excitaciones generales con amortiguación
### Cálculo directo de la integral de Duhamel
#### Fuerza Constante


#### Problema 5.
Considerar una torre sometida a una aceleración constante de $0.5g$ aplicada durante 0.5 segundos a sus cimientos. Determinar la respuesta de la torre en función del desplazamineto y de la velocidad de la masa relativos al movimiento del cimiento. Determinar también la aceleración máxima de la masa. Suponer la amortiguación igual al $20\%$ de la mortiguación crítica.

In [1]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
from librerias.integral import cuad_gauss # librería que aproxima las integrales mediante el método de Cuadratura de Gauss

In [2]:
# Iniciamos las variables
P = 98000 # Peso W en kp
g = 980 # Gravedad en cm/s2
m = P/g  # masa en kp*s2/cm
k = 100000  # rigidez en kp/cm
e = 0.2 # razón de amortiguación del sistema

F = F = lambda t: -m * g * 0.5	# Fuerza con aceleración constante aplicada al cimiento
tf = 0.6 # tienpo hasta que se aplica la fuerza
dt = 0.02 # intervalos de tiempo

Cálculos:

In [3]:
w = (k/m)**0.5  # Frecuencia angular natural(rad/s)
wd = w*(1 - e**2)**0.5 # Frecuencia del sistema con amortiguación

Utilizaremos las siguientes ecuaciones:
$$ A_D(t_i)=A_D(t_{i-1}) + \int^{t_i}_{t_{i-1}} F(\tau) e^{\xi \omega \tau}cos(\omega_{D}\tau) d\tau$$
$$ B_D(t_i)=B_D(t_{i-1}) + \int^{t_i}_{t_{i-1}} F(\tau) e^{\xi \omega \tau}sen(\omega_{D}\tau) d\tau$$


$$ y(t_i) = \frac{e^{\xi \omega t_i}}{m \omega_D}(A_D(t_i)sen(\omega_D t_i)-B_D(t_i)cos(\omega_D t_i) )$$

In [4]:
# Variables iniciales necesarias
y0 = 0 # desplazamiento inicial
v0 = 0 # velocidad inicial
a0 = 0 # acelereación inicial
A = 0
B = 0
AA = [A]
BB = [B]
y = [y0]
v = [v0]
a = [a0]
n = int(tf / dt) 
tt = np.arange(0, tf + dt, dt) # vector con los intervalos de tiempo

In [5]:
# definimos las ecuaciones dentro de las integrales
eqA = lambda t: F(t) * np.exp(e * w * t) * np.cos(wd * t)
eqB = lambda t: F(t) * np.exp(e * w * t) * np.sin(wd * t)

In [6]:
# definimos las función y(t) = f y derivadas
f = lambda t: (A*np.sin(wd * t) - B * np.cos(wd * t)) * np.exp(-e * w * t) / (m * wd)
h = 0.000001
df = lambda t: (f(t + h) - f(t))/h # primera derivada
ddf = lambda t: (df(t + h) - df(t))/h # segunda derivada

In [7]:
# iteramos
for i in range(n):
    A = A + cuad_gauss(tt[i], tt[i + 1], eqA)
    B = B + cuad_gauss(tt[i], tt[i + 1], eqB)
    
    AA.append(A)
    BB.append(B)
    y.append(f(tt[i + 1]))
    v.append(df(tt[i + 1]))
    a.append(ddf(tt[i + 1]))

In [8]:
df = pd.DataFrame({'t(s)':tt,'A':AA,'B':BB,'y':y,'v':v,'a':a})
df


,t(s),A,B,y,v,a
0,0.00,0.00000,0.00000,0.000000,0.000000,0.000000
1,0.02,-977.02275,-319.88801,-0.087316,-8.093326,189.696273
2,0.04,-1668.79669,-1259.35103,-0.292753,-11.611253,439.633996
3,0.06,-1688.69640,-2583.19572,-0.519691,-10.374033,650.919207
4,0.08,-834.55621,-3819.32405,-0.685755,-5.867413,759.973529
5,0.10,769.23935,-4398.35173,-0.747792,-0.362532,752.373053
6,0.12,2632.50784,-3876.24198,-0.707849,4.035905,656.789734
7,0.14,4009.74473,-2165.84969,-0.602372,6.071817,525.560040
8,0.16,4154.79165,321.98302,-0.482063,5.577394,411.505996
9,0.18,2649.10141,2715.92554,-0.391358,3.287207,349.772156


In [16]:
print("Desplazamiento máximo: {}\nVelocidad máxima: {}\nAceleración máxima: {}".format(round((df['y'].abs()).max(),3),round((df['v'].abs()).max(),3),round((df['a'].abs()).max(),3)))

Desplazamiento máximo: 0.748
Velocidad máxima: 11.611
Aceleración máxima: 759.974
